
## Crisis Prediction in African Economies  

Given *data about financial climates in various African countries throughout the years*, let's try to predict whether a **banking crisis** occurred or not in a given year in a given country.

We will use a TensorFlow ANN to make our predictions.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
data = pd.read_csv('../input/africa-economic-banking-and-systemic-crisis-data/african_crises.csv')

In [3]:
data

,case,cc3,country,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises,banking_crisis
0,1,DZA,Algeria,1870,1,0.052264,0,0,0.0,3.441456,0,0,0,crisis
1,1,DZA,Algeria,1871,0,0.052798,0,0,0.0,14.149140,0,0,0,no_crisis
2,1,DZA,Algeria,1872,0,0.052274,0,0,0.0,-3.718593,0,0,0,no_crisis
3,1,DZA,Algeria,1873,0,0.051680,0,0,0.0,11.203897,0,0,0,no_crisis
4,1,DZA,Algeria,1874,0,0.051308,0,0,0.0,-3.848561,0,0,0,no_crisis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,70,ZWE,Zimbabwe,2009,1,354.800000,1,1,0.0,-7.670000,1,1,0,crisis
1055,70,ZWE,Zimbabwe,2010,0,378.200000,1,1,0.0,3.217000,1,0,0,no_crisis
1056,70,ZWE,Zimbabwe,2011,0,361.900000,1,1,0.0,4.920000,1,0,0,no_crisis
1057,70,ZWE,Zimbabwe,2012,0,361.900000,1,1,0.0,3.720000,1,0,0,no_crisis


Checking if systemic crisis differs from banking crisis in any way.

In [4]:
((data['banking_crisis'] == 'crisis').astype(int) == data['systemic_crisis']).all()

False

Check for missing values

In [5]:
data.isna().sum()

case                               0
cc3                                0
country                            0
year                               0
systemic_crisis                    0
exch_usd                           0
domestic_debt_in_default           0
sovereign_external_debt_default    0
gdp_weighted_default               0
inflation_annual_cpi               0
independence                       0
currency_crises                    0
inflation_crises                   0
banking_crisis                     0
dtype: int64

## Encoding

In [6]:
data = data.drop(['case', 'country'], axis=1)

In [7]:
data

,cc3,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises,banking_crisis
0,DZA,1870,1,0.052264,0,0,0.0,3.441456,0,0,0,crisis
1,DZA,1871,0,0.052798,0,0,0.0,14.149140,0,0,0,no_crisis
2,DZA,1872,0,0.052274,0,0,0.0,-3.718593,0,0,0,no_crisis
3,DZA,1873,0,0.051680,0,0,0.0,11.203897,0,0,0,no_crisis
4,DZA,1874,0,0.051308,0,0,0.0,-3.848561,0,0,0,no_crisis
...,...,...,...,...,...,...,...,...,...,...,...,...
1054,ZWE,2009,1,354.800000,1,1,0.0,-7.670000,1,1,0,crisis
1055,ZWE,2010,0,378.200000,1,1,0.0,3.217000,1,0,0,no_crisis
1056,ZWE,2011,0,361.900000,1,1,0.0,4.920000,1,0,0,no_crisis
1057,ZWE,2012,0,361.900000,1,1,0.0,3.720000,1,0,0,no_crisis


In [8]:
cc3_dummies = pd.get_dummies(data['cc3'])
data = pd.concat([data, cc3_dummies], axis=1)
data = data.drop('cc3', axis=1)

In [9]:
data

,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises,...,DZA,EGY,KEN,MAR,MUS,NGA,TUN,ZAF,ZMB,ZWE
0,1870,1,0.052264,0,0,0.0,3.441456,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1871,0,0.052798,0,0,0.0,14.149140,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1872,0,0.052274,0,0,0.0,-3.718593,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1873,0,0.051680,0,0,0.0,11.203897,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1874,0,0.051308,0,0,0.0,-3.848561,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,2009,1,354.800000,1,1,0.0,-7.670000,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1055,2010,0,378.200000,1,1,0.0,3.217000,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1056,2011,0,361.900000,1,1,0.0,4.920000,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1057,2012,0,361.900000,1,1,0.0,3.720000,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
y = data['banking_crisis']
X = data.drop('banking_crisis', axis=1)

In [11]:
y

0          crisis
1       no_crisis
2       no_crisis
3       no_crisis
4       no_crisis
          ...    
1054       crisis
1055    no_crisis
1056    no_crisis
1057    no_crisis
1058    no_crisis
Name: banking_crisis, Length: 1059, dtype: object

In [12]:
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)
{index: label for index, label in enumerate(label_encoder.classes_)}

{0: 'crisis', 1: 'no_crisis'}

In [13]:
y = pd.Series(y).apply(lambda x: 1 - x)

Now `y=1` means we have a crisis and `y=0` means we do not.

## Scaling

In [14]:
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [15]:
X

,year,systemic_crisis,exch_usd,domestic_debt_in_default,sovereign_external_debt_default,gdp_weighted_default,inflation_annual_cpi,independence,currency_crises,inflation_crises,...,DZA,EGY,KEN,MAR,MUS,NGA,TUN,ZAF,ZMB,ZWE
0,-2.917150,3.451758,-0.386713,-0.203219,-0.424973,-0.147006,-0.030863,-1.862352,-0.378058,-0.385474,...,3.385088,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,-0.304761
1,-2.887313,-0.289707,-0.386708,-0.203219,-0.424973,-0.147006,-0.030848,-1.862352,-0.378058,-0.385474,...,3.385088,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,-0.304761
2,-2.857475,-0.289707,-0.386712,-0.203219,-0.424973,-0.147006,-0.030874,-1.862352,-0.378058,-0.385474,...,3.385088,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,-0.304761
3,-2.827638,-0.289707,-0.386718,-0.203219,-0.424973,-0.147006,-0.030852,-1.862352,-0.378058,-0.385474,...,3.385088,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,-0.304761
4,-2.797800,-0.289707,-0.386721,-0.203219,-0.424973,-0.147006,-0.030874,-1.862352,-0.378058,-0.385474,...,3.385088,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,-0.304761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,1.230271,3.451758,2.797088,4.920801,2.353091,-0.147006,-0.030880,0.536955,2.481682,-0.385474,...,-0.295413,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,3.281260
1055,1.260109,-0.289707,3.007099,4.920801,2.353091,-0.147006,-0.030864,0.536955,-0.378058,-0.385474,...,-0.295413,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,3.281260
1056,1.289946,-0.289707,2.860809,4.920801,2.353091,-0.147006,-0.030861,0.536955,-0.378058,-0.385474,...,-0.295413,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,3.281260
1057,1.319784,-0.289707,2.860809,4.920801,2.353091,-0.147006,-0.030863,0.536955,-0.378058,-0.385474,...,-0.295413,-0.414078,-0.259885,-0.276079,-0.26195,-0.245072,-0.276079,-0.347325,-0.270089,3.281260


## Splitting

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# Training

In [17]:
X.shape

(1059, 23)

In [18]:
y.sum() / len(y)

0.08876298394711993

In [19]:
inputs = tf.keras.Input(shape=(23,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name="auc")]
)


batch_size = 64
epochs = 60

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau()],
    verbose=0
)

# Results

In [20]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss"
)

fig.show()

In [21]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 1ms/step - loss: 0.1147 - auc: 0.9560


[0.11465290933847427, 0.95600426197052]